In [15]:
import os
import pandas as pd
from openai import OpenAI

# Read data
# train_folder = "dataset/AdMIRe Subtask A Train/"
# train_df_file_path = train_folder + "subtask_a_train_gpt-4_meanings.tsv"

# train_folder = "dataset/AdMIRe Subtask A Train/"
# train_df_file_path = train_folder + "subtask_a_train_gpt-4o_meanings.tsv"

# train_folder = "dataset/AdMIRe Subtask A PT Train/"
# train_df_file_path = train_folder + "subtask_a_train_gpt-4_meanings.tsv"

train_folder = "dataset/AdMIRe Subtask A PT Train/"
train_df_file_path = train_folder + "subtask_a_train_gpt-4o_meanings.tsv"

df = pd.read_csv(train_df_file_path, sep='\t')

# Initialize the OpenAI client
api_key = ""   # <================= Change the API key here
client = OpenAI(api_key=api_key)

# Define a function to get a response from the LLM
def get_gpt_response(message):
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": message,
            }
        ],
        model="gpt-4",
    )
    response_content = chat_completion.choices[0].message.content.strip()
    return response_content

# Define a function to generate idiomatic meanings
def generate_idiomatic_meaning(sentence, compound):
    query = (
        f"Given an example sentence: {sentence}\n"
        f"What is the idiomatic meaning of '{compound}' in this example sentence?\n"
        "Provide one answer without accompanying texts."
    )
    return get_gpt_response(query)

# Filter rows where `meaning` needs to be updated
missing_meanings = df[(df['sentence_type'] == 'idiomatic') & (df['sent_type_predicted'] == 'literal')]

# Generate idiomatic meanings and replace values in the `meaning` column
for index, row in missing_meanings.iterrows():
    sentence = row['sentence']
    compound = row['compound']
    meaning = generate_idiomatic_meaning(sentence, compound)
    if meaning:
        df.at[index, 'meaning'] = meaning  # 替换 `meaning` 列中的值

# Save the updated data
output_file_path = train_df_file_path.replace(".tsv", "_updated.tsv")
df.to_csv(output_file_path, sep='\t', index=False)
print(f"Updated file saved to: {output_file_path}")

Updated file saved to: dataset/AdMIRe Subtask A PT Train/subtask_a_train_gpt-4o_meanings_updated.tsv
